1. remove useless columns
2. filters from params.yaml

In [90]:
import polars as pl
import yaml


In [ ]:
with open("../params.yaml", "r") as file:
    params = yaml.safe_load(file)
input_file_path = params["clean"]["input_data_path"]

# read in the complete data frame
df = pl.scan_parquet("../" + input_file_path)

df = df.with_columns(pl.col("game_date").str.to_datetime())
df = df.filter(pl.col("game_date").dt.year() >= params["clean"]["start_year"])

In [92]:
# df = df.with_columns(
#     df.select(pl.col("pitch_type").cast(pl.Categorical)),
# )
# df.head()


df = df.drop(
    [
        "Unnamed: 0",
        "spin_dir",
        "spin_rate_deprecated",
        "break_angle_deprecated",
        "break_length_deprecated",
        "game_type",
        "home_team",
        "away_team",
        "des",
        "description",
        "game_year",
        "tfs_deprecated",
        "tfs_zulu_deprecated",
        "fielder_2",
        "umpire",
        "sv_id",
        "pitcher.1",
        "fielder_2.1",
        "fielder_3",
        "fielder_4",
        "fielder_5",
        "fielder_6",
        "fielder_7",
        "fielder_8",
        "fielder_9",
        "pitch_name",
        "home_score",
        "away_score",
        "post_away_score",
        "post_home_score",
        "post_bat_score",
        "post_fld_score",
        "delta_home_win_exp",
        "delta_run_exp",
    ]
)


In [99]:
df = df.filter(pl.col("game_date").dt.year() >= params["clean"]["start_year"])

df.filter(pl.col("pitcher").is_not_null())
df.filter(pl.col("pitch_type").is_not_null())

# get top k pitchers (decided by number of pitches and num_pitchers from params.yaml)
pitcher_counts = df.group_by("pitcher").len().sort("len", descending=True)
top_k_pitchers = pitcher_counts.head(params["clean"]["num_pitchers"]).select("pitcher")

# Collect the 'pitcher' column into a list
top_k_pitchers_list = top_k_pitchers.collect().get_column("pitcher").to_list()

# Now 'is_in' can use the list
df = df.filter(pl.col("pitcher").is_in(top_k_pitchers_list))

In [100]:
def display_df(df):
    with pl.Config() as cfg:
        cfg.set_tbl_cols(df.width)
        display(df)

In [104]:
display_df(df.collect())

pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,zone,stand,p_throws,type,hit_location,bb_type,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,bat_speed,swing_length
str,datetime[μs],f64,f64,f64,str,i64,i64,str,f64,str,str,str,f64,str,i64,i64,f64,f64,f64,f64,f64,f64,f64,i64,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,str,str,f64,f64,f64
"""ST""",2024-10-20 00:00:00,78.8,4.1,5.27,"""Manaea, Sean""",571970,640455,"""walk""",14.0,"""L""","""L""","""B""",null,null,3,1,-1.04,0.07,1.44,1.65,null,null,null,0,3,"""Bot""",null,null,-3.876776,-114.790473,-1.213129,-8.519711,20.865706,-31.531093,2.99,1.5,null,null,null,79.7,2141.0,6.8,775302,53.72,null,0.689131,0.7,1.0,0.0,0.0,null,31,5,4,1,"""Infield shade""","""Standard""",294.0,null,null
"""CH""",2024-10-20 00:00:00,83.9,4.16,5.13,"""Manaea, Sean""",571970,640455,null,13.0,"""L""","""L""","""B""",null,null,2,1,1.28,-0.01,-1.13,1.32,null,null,null,0,3,"""Bot""",null,null,-14.910726,-121.300964,-2.306161,15.532788,23.944392,-32.030777,2.99,1.4,null,null,null,84.7,1835.0,7.2,775302,53.31,null,null,null,null,null,null,null,31,4,4,1,"""Infield shade""","""Standard""",106.0,null,null
"""SI""",2024-10-20 00:00:00,91.9,4.03,4.97,"""Manaea, Sean""",571970,640455,null,13.0,"""L""","""L""","""B""",null,null,1,1,1.16,0.76,-0.93,2.17,null,null,null,0,3,"""Bot""",null,null,-15.210428,-132.944842,-2.837582,16.955867,27.981449,-22.734943,2.99,1.53,null,null,null,92.7,2131.0,7.0,775302,53.48,null,null,null,null,null,null,null,31,3,4,1,"""Infield shade""","""Standard""",113.0,null,null
"""ST""",2024-10-20 00:00:00,80.2,3.84,5.31,"""Manaea, Sean""",571970,640455,null,13.0,"""L""","""L""","""B""",null,null,0,1,-1.34,0.15,-0.86,1.28,null,null,null,0,3,"""Bot""",null,null,-7.987162,-116.480556,-2.438514,-10.452965,23.959752,-30.532682,2.99,1.35,null,null,null,81.3,2117.0,7.4,775302,53.14,null,null,null,null,null,null,null,31,2,4,1,"""Infield shade""","""Standard""",284.0,null,null
"""ST""",2024-10-20 00:00:00,78.9,3.89,5.26,"""Manaea, Sean""",571970,640455,null,8.0,"""L""","""L""","""S""",null,null,0,0,-1.57,0.19,-0.23,1.53,null,null,null,0,3,"""Bot""",null,null,-6.125394,-114.782271,-1.669025,-12.671426,21.628193,-30.380521,2.99,1.48,null,null,null,79.9,2152.0,7.0,775302,53.45,null,null,null,null,null,null,null,31,1,4,1,"""Infield shade""","""Standard""",283.0,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
null,2024-03-16 00:00:00,null,null,null,"""Peralta, Freddy""",665750,642547,"""field_out""",null,"""L""","""R""","""X""",6.0,"""ground_ball""",1,2,null,null,null,null,null,null,null,0,1,"""Top""",117.74,155.77,null,null,null,null,null,null,null,null,null,null,null,null,null,null,747933,null,null,null,0.0,null,0.0,0.0,null,1,5,0,0,null,null,null,null,null
null,2024-03-16 00:00:00,null,null,null,"""Peralta, Freddy""",665750,642547,null,null,"""L""","""R""","""S""",null,null,1,2,null,null,null,null,null,null,null,0,1,"""Top""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,747933,null,null,null,null,null,null,null,null,1,4,0,0,null,null,null,null,null
null,2024-03-16 00:00:00,null,null,null,"""Peralta, Freddy""",665750,642547,null,null,"""L""","""R""","""B""",null,null,0,2,null,null,null,null,null,null,null,0,1,"""Top""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,747933,null,null,null,null,null,null,null